In [2]:
#Sktime imports
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sktime.classification.shapelet_based import MrSEQLClassifier
from sklearn.model_selection import GroupShuffleSplit

from IPython.utils import io
import pandas as pd
import os
from pathlib import Path
from matplotlib import pyplot as plt
from scipy.signal import find_peaks
import numpy as np
import seaborn as sns
import time
import joblib
import sys
from os.path import exists

import importlib

# EEG package modules__________________________________________________________________
import constants
import custom_ts_length
# import eeg_stat_ts
import format_eeg_data
import run_expts

from sklearn.metrics import accuracy_score

In [3]:
'''
o 1. PD vs HC,   2.(PD , PD+RBD) vs HC,   3.(PD + PD&RBD) vs HC vs RBD  multiclass, 4.PD vs PD+RBD vs RBD vs HC 
'''
core_path = '/export/sphere/ebirah/ko20929/RBD_files/notebooks/frequency_analysis/constructed_data/band_power_time_series/expt_dfs/'

#1 - This is to run the custom length expts
folders = {'N1': 'N1_customised_40_expt_files/' , 'N2' : 'N2_120_expt_files_v2/','N3' : 'N3_82_expt_files/', 'REM' :'REM_19_expt_files/'}
#2 -This is to run the expts on the full original length data without any dropping or cutting of the TS 
folders = {'N1': 'N1_full_expts/' , 'N2' : 'N2_full_expts/','N3' : 'N3_full_expts/', 'REM' :'REM_full_expts/'}
#3 - This is to run the wakeful eyes closed experiments
folders = {'EC1': 'EC_8_expts/' , 'EC2' : 'EC_full_expts/' }

#Folder = folders[sys.argv[1]] #User can input which experiments will be run
print('Change the folder here ^_^! ')

folder = folders['EC1'] #I'm choosing this because it's the TS & Therefore quickest hopefully?
folder = 'REM_full_expts/'

expts = ['MrSQL_expt_1', 'MrSQL_expt_2' , 'MrSQL_expt_3' , 'MrSQL_expt_4']

power_band_cols = ['Delta', 'Theta', 'Alpha', 'Sigma', 'Beta', 'Gamma', 'TotalAbsPow']

regions = ['Prefrontal','Frontal', 'Left Frontal', 'Right Frontal', 'Central', 'Left Central', 
           'Right Central', 'Left Temporal','Right Temporal', 'Parietal', 'Left Parietal', 'Right Parietal', 'Occipital']

#Load the appropriate X (full set of features ) , y (full set of labels) , groups 
load_path = core_path + folder
X = pd.read_hdf(load_path + 'X.h5', key='df', mode='r')
y = pd.read_hdf(load_path + 'y_full.h5', key='df', mode='r') 
groups = pd.read_hdf(load_path + 'groups.h5', key='df', mode='r')  

Change the folder here ^_^! 


In [4]:
#1.Generate expt specific X,y,groups
X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups,1)

In [7]:
X_Delta = X_expt[['Delta_Prefrontal']]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_Delta, y_expt, random_state = 43)

### Train classifier on Test data vs Train & test data , look at params

In [6]:
classifier = MrSEQLClassifier(seql_mode='fs', symrep=[ 'sfa' , 'sax']) 

In [9]:
%%time
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
accuracy_score(y_test, y_pred)

CPU times: user 2.33 s, sys: 18.7 ms, total: 2.35 s
Wall time: 2.41 s


7e-01

In [10]:
classifier.fit(X_test, y_test)
y_pred = classifier.predict(X_test)
accuracy_score(y_test, y_pred)

1e+00

In [12]:
y_pred = classifier.predict(X_train)
accuracy_score(y_train, y_pred)

6e-01

### Train on Test only

In [11]:
classifier_2 = MrSEQLClassifier(seql_mode='fs', symrep=[ 'sfa' , 'sax']) 

classifier_2.fit(X_test, y_test)
y_pred = classifier_2.predict(X_test)
accuracy_score(y_test, y_pred)

1e+00

In [13]:
y_pred = classifier_2.predict(X_train)
accuracy_score(y_train, y_pred)

6.333333333333333e-01

In [14]:
## X train , X_train 

In [15]:
classifier_3 = MrSEQLClassifier(seql_mode='fs', symrep=[ 'sfa' , 'sax']) 

classifier_3.fit(X_train, y_train)
classifier_3.fit(X_train, y_train)

y_pred = classifier_3.predict(X_test)
accuracy_score(y_test, y_pred)

7e-01

### Try to look at the model parameters

In [35]:
len( classifier.seql_clf.features[3] )

28

In [36]:
len( classifier_2.seql_clf.features[0] )

28

In [37]:
 classifier_2.seql_clf.features[0]

[[b'aaabb',
  b'aadba',
  b'aadbc',
  b'aaddb',
  b'ababa',
  b'abdc',
  b'acdd',
  b'adbc',
  b'adbcd',
  b'adbd',
  b'adbdd',
  b'addbb',
  b'baacb',
  b'babaa',
  b'babad',
  b'babbc',
  b'bacd',
  b'bcad',
  b'bccdb',
  b'bdbdb',
  b'caad',
  b'cadb',
  b'cbdbd',
  b'ccaac',
  b'ccbac',
  b'ccbdd',
  b'cccbc',
  b'ccccb',
  b'cccccd',
  b'ccccd',
  b'cccdb',
  b'ccdda',
  b'cdccc',
  b'cdda',
  b'dbac',
  b'dbcdd',
  b'ddbbc'],
 [b'aaccb',
  b'abaa',
  b'abbdc',
  b'abcbbc',
  b'abcccd',
  b'abccdcc',
  b'abcdd',
  b'acbbd',
  b'addcc',
  b'baacbc',
  b'bbbab',
  b'bbda',
  b'bbdccbc',
  b'bcabb',
  b'bcbbcc',
  b'bcbdcd',
  b'bccbd',
  b'bcccaa',
  b'bcccd',
  b'bcccdc',
  b'bcdbb',
  b'bcdca',
  b'bcdcd',
  b'bdcccd',
  b'cabbd',
  b'cadbb',
  b'cbdccb',
  b'ccbba',
  b'ccbccbb',
  b'cccaa',
  b'cccdd',
  b'cccddd',
  b'ccddd',
  b'cdbbcb',
  b'cdcca',
  b'dbbccd',
  b'dbccc',
  b'dcccd',
  b'ddcbb',
  b'ddccbb'],
 [b'abbcc',
  b'abccbcc',
  b'abcccd',
  b'abdcc',
  b'abdccc',
  